## Below is the same code as Example_1_toymodel.py
## It is broken out into a Jupyter Notebook 

In [1]:
import mfapy

In [4]:
# Model construction
reactions, reversible, metabolites, fragments\
 = mfapy.mfapyio.load_metabolic_model("example_1_toymodel_model.txt")
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible,\
 metabolites, fragments)

Error!: Metabolite Sucex is not used in the metabolic network, but produced  in reaction(s)  ['v6']
There are critical problem(s) in the metabolic model


In [ ]:
# Addition of constraints
state = model.load_states("Example_1_toymodel_status.csv", format = 'csv')
model.set_constraints_from_state_dict(state)
model.update()

In [ ]:
# Generation of CarbonSource instance
cs1 = model.generate_carbon_source_template()
cs1.set_each_isotopomer('AcCoA', {'#10':0.5})
cs2 = model.generate_carbon_source_template()
cs2.set_each_isotopomer('AcCoA', {'#11':0.5})

In [ ]:
# Load MDV data
mdv1 = model.load_mdv_data("Example_1_MDV1.txt")
mdv2 = model.load_mdv_data("Example_1_MDV2.txt")

In [ ]:
# Flux estimation Step 1: Setting experimments
model.set_experiment('ex1', mdv1, cs1)
model.set_experiment('ex2', mdv2, cs2)

In [ ]:
# Flux estimation step 2: Generation of intical flux vectors
endstate, flux = model.generate_initial_states(50, 4, method ="parallel")

In [ ]:
# Flux estimation step 3: Fitting model
for method in ["GN_CRS2_LM", "LN_PRAXIS", "SLSQP"]:
    endstate, RSS, flux = model.fitting_flux(method = method, flux = flux)
model.show_results([("final", flux[0])]) # Show result

In [ ]:
# Estimation of 95% CI
ci_edge = model.generate_ci_template(targets = [('reaction', "v3")])
ci = model.search_ci(ci_edge, flux[0], method = 'grid')
lb = ci['data'][('reaction', "v3")]['lower_boundary']
ub = ci['data'][('reaction', "v3")]['upper_boundary']
print("v3", "Lower bondary:",lb, "Upper boundary:", ub)
